In [9]:
from transformers import AutoModelForCausalLM, pipeline
import torch
import time
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain

inicio = time.time()

# Number of times to execute the code
num_executions = 5

# Output file path
output_file_path = "output_results_3.txt"

# Loop for multiple executions
for i in range(num_executions):
    print(f"Execution {i+1}/{num_executions}")
    
    context = f"""
    Requirement 1: The system must implement a secure user authentication process, \
    including username and password verification, to ensure access control and protect user account. \
    Requirement 2: The system should provide a password recovery mechanism that allows users to reset their passwords in case they forget their login credentials.
    """

    template = """   

    I will present a pair of software requirements in the context delimited by angle brackets. There may be contradictions between these requirements. \
    
    A contradiction in software requirements refers to a scenario in which two or more specified \
    requirements are mutually exclusive or incompatible.
    
    Provide a response to the question and offer a detailed explanation to support your answer using a maximum of 100 words. 
    
    If you don't have the information then give response \"I don't know\".

    Context:
    <{context}>

    Question: {question}

      Response:
      """

    def build_qa_chain():
        torch.cuda.empty_cache()
        model_name = "databricks/dolly-v2-3b"

        instruct_pipeline = pipeline(model=model_name, 
                                 torch_dtype=torch.bfloat16,
                                 device_map="auto",
                                 return_full_text=True,
                                 max_new_tokens=256,
                                 top_p=0.95,
                                 top_k=50,
                                 trust_remote_code=True 
                                # model_kwargs={'load_in_8bit': True}
                                )

        model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                     device_map="auto",
                                                     torch_dtype=torch.float16)

        return instruct_pipeline, model

    instruct_pipeline, model = build_qa_chain()


    question = "Do these requirements contradict each other?"

    response = instruct_pipeline(template.format(context=context, question=question))
    
    # Extract the response from the output
    response_text = response[0]['generated_text'].split('Response:')[1].strip()
    print(response_text)

     # Print and save the output to the file
    with open(output_file_path, "a") as output_file:
        output_file.write(f"Execution {i+1}/{num_executions}:\n")
        output_file.write(response_text)
        output_file.write("\n\n")

print("All executions completed.")

fin = time.time()
print((fin - inicio)/60)

Execution 1/5
Both requirements 1 and 2 can be satisfied without compromising requirement 1. 

In other words, the requirements are not mutually exclusive or incompatible.
Execution 2/5
No, the requirements are not contradictory. Both requirements describe desirable features that the system should have. 
  
    
Requirement 1: The system must implement a secure user authentication process, including username and password verification, to ensure access control and protect user account. 
  
    The requirement describes a desirable feature. 
  
    
Requirement 2: The system should provide a password recovery mechanism that allows users to reset their passwords in case they forget their login credentials. 
  
    The requirement describes a desirable feature. 
  
    
Providing a password recovery mechanism is a desirable and desirable feature. However, it does not provide a secure user authentication process, including username and password verification. Therefore, the requirements are 